In [2]:
mkdir test

In [3]:
cd test/

/home/bercherj/JFB/Python/ElementsAutourDeFlask/test


In [5]:
#%%file config.py
# Sera utilisé pour configurer la db
basedir  = os.getcwd()
basedir = basedir[0] if isinstance(basedir, list) else basedir

class Config(object):
    # ...
    SQLALCHEMY_DATABASE_URI = os.environ.get('DATABASE_URL') or \
        'sqlite:///' + os.path.join(basedir, 'app.db')
    SQLALCHEMY_TRACK_MODIFICATIONS = False

Writing config.py


In [6]:
ls

config.py


In [7]:
!mkdir -p exo

In [8]:
#%%file exo/__init__.py
from flask import Flask
#from config import Config # si la config est définie dans config.py
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config.from_object(Config)
db = SQLAlchemy(app)


#from app import routes, models 

Writing exo/__init__.py


In [9]:
%%file exo/models.py
from datetime import datetime
from app import db  # si app définie dans le __init__

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), index=True, unique=True)
    email = db.Column(db.String(120), index=True, unique=True)
    password_hash = db.Column(db.String(128))
    posts = db.relationship('Post', backref='author', lazy='dynamic') # crée un lien avec un attribut "author" pour chacun des posts

    def __repr__(self):
        return '<User {}>'.format(self.username)

class Years(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    body = db.Column(db.String(140))
    timestamp = db.Column(db.DateTime, index=True, default=datetime.utcnow)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))

    def __repr__(self):
        return '<Year {}>'.format(self.body)
    
  

Writing exo/models.py


In [10]:
%%file exo/forms.py

from flask_wtf import Form
from wtforms import StringField, SubmitField, PasswordField, BooleanField
from wtforms.validators import DataRequired

class LoginForm(Form):
    login = StringField('Login', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    remember_me = BooleanField('Remember me', default=False)
    submit = SubmitField('Go')

Writing exo/forms.py


In [13]:
mkdir exo/templates

In [14]:
%%file exo/templates/login.html
{% extends "bootstrap/base.html" %}

{% block head %}
    {{ super() }}
        <title>{% block title %}
        {% if title %}
            {{ title }} 
        {% else %}
            Page
        {% endif %}
    {% endblock title %}</title>
    <style> .container {
        margin-left: 30px;
    } 
    </style>


{% endblock %}


{% block content %}

    
<div class="container">

    {% for message in get_flashed_messages() %}
    <div class="alert alert-warning">
        <button type="button" class="close" data-dismiss="alert">&times;</button>
        {{ message }}
    </div>
    {% endfor %}

    {% block page_content %}{% endblock %}
    {% block container %} {% endblock %}




<form method="POST" action="/login">
{{ form.hidden_tag() }}
<p>
  <b>{{ form.login.label }} :</b><br>
  {{ form.login(size=40, autofocus=True) }}<br>
  {% for error in form.login.errors %}
  <span style="color: red;">[{{ error }}]</span>
  {% endfor %}<br>
</p>

<p>
  <b>{{ form.password.label }} :</b><br>
  {{ form.password(size=40) }}<br>
  {% for error in form.password.errors %}
  <span style="color: red;">[{{ error }}]</span>
  {% endfor %}<br>
</p>

<p>{{ form.submit }}</p>

<p>{{ form.remember_me }} <b>{{ form.remember_me.label }}</b></p>

</form>

</div>

{% endblock %}

Writing exo/templates/login.html


In [15]:
%%file run.py
from flask import Flask, flash, redirect, render_template, \
     request, url_for
from flask_login import LoginManager, logout_user, login_required, \
       login_user, current_user, UserMixin
from ldap3 import Connection, ALL, Server

from forms import LoginForm    
from flask_bootstrap import Bootstrap

UPLOAD_FOLDER = '/tmp/'
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])


app = Flask(__name__)
app.config['SECRET_KEY'] = 'you-will-never-guess'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024 # 16 Mb

Bootstrap(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
login_manager.login_message = 'Veuillez vous connecter pour accéder à cette page.'


if __name__=='__main__':
    app.run(debug=True, port=2747)


Writing run.py


In [16]:
%%file exo/views.py
#!/usr/bin/env python


##
def connect(accountName,password):
    server = Server('dc1.lan.esiee.fr', use_ssl=True, get_info=ALL)
    conn = Connection(server, user="cn=LDAP,ou=comptes_services,ou=utilisateurs,DC=lan,DC=esiee,DC=fr",password="UE=cv,VR1^%Mbj43")
    conn.bind()
    conn.search('DC=lan, DC=esiee, DC=fr', "(&(objectclass=person)(sAMAccountName="+accountName+"))",attributes=['distinguishedName', 'sn', 'telephoneNumber', 'displayName', 'roomNumber', 'givenName','Name'])
    if len(conn.entries)>0:
        DN = conn.entries[0].distinguishedName.value
        conn = Connection(server, user=DN, password=password) 
        if(conn.bind()) :
            return True
    return False
##

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route('/')
def rien():
    return "Rien ici... <a href='/login'> voir là </a> "

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = LoginForm()
    if form.validate_on_submit():
        if connect(form.login.data, form.password.data):
            u = User(42, form.login.data)
            login_user(u, remember=form.remember_me.data)
            return redirect('/success/'+form.login.data)
    return render_template('login.html', form=form)

@login_required
@app.route('/success/<username>')
def sucess(username):
    return "Salut " + username + ".\nTu es arrivé jusque là. "


@app.route('/upload', methods = ['GET', 'POST'])
@app.route('/uploader', methods = ['GET', 'POST'])
def upload_file():
    import pandas as pd
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename): 
            filename = secure_filename(file.filename)
            uploaded_file = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(uploaded_file, filename))
            flash('file uploaded successfully')        
            """
            Ici ajouter le traitement du fichier
            year = filename.split('_')[-1]
            df = pd.read_csv(uploaded_file)
            for worker in df.index:
                u = User(worker, year, *df.loc[worker,:]) 
                db.session.add(u)
                db.session.commit()
            """
            return redirect(url_for('index'))
        else:
            if file and not allowed_file(file.filename): 
                flash("Un fichier de type autorisé svp")
    return render_template('upload.html', allowed=ALLOWED_EXTENSIONS)  

@app.route('/download/<file>')
def download(file):
    ext = file.split('.')[-1]
    return send_file(file,
                     attachment_filename="result."+ext,
                     as_attachment=True)

@app.route('/plot')


@login_required
@app.route('/logout')
def logout():
    logout_user()
    return "you are logged out"



Writing exo/views.py


In [17]:
%%file exo/templates/upload.html
<html>
    {% with messages = get_flashed_messages() %}
      {% if messages %}
        {% for message in messages %}
          {{ message }}
        {% endfor %}
      {% endif %}
    {% endwith %}
   <body>
      <h2> File upload </h2>
      <p> Choisir un fichier de type {{ allowed }} </p>
      <form action = "/uploader" method = "POST" 
         enctype = "multipart/form-data">
         <input type = "file" name = "file" /> <br><br>
         <input type = "submit"/>
      </form>
      
   </body>
</html>

Writing exo/templates/upload.html
